In [ ]:
from marked_words import marked_words
import pandas as pd
from collections import Counter

def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        #print("%s, %.2f" % (word[0],word[1]))
        full_list.append(word[0])
    return full_list


In [ ]:
models=["4o","4v","Gemini","LLaVA","llama","Midjourney"]

In [ ]:
count_race_lst = {}
count_gender_lst = {}
all_mw={}
for model in models:
    df = pd.read_csv(f'../../result/image_description/{model}/all_data.csv')
    df["text"]=df['text'].apply(lambda x: x.rstrip(":1234567890 "))

    mw = {}
    for race in df['race'].unique():
        outs = pprint(marked_words(df, [race], ['race'],['white']))
        mw[race] = outs
    temps = []
    for race in df['race'].unique():
        temp = pprint(marked_words(df, ['white'], ['race'],[race]))
        temps.extend(temp)
    seen = Counter(temps).most_common()

    temp=[w for w, c in seen if c == 2]
    mw['white']=temp[:10]


    for race in df['gender'].unique():
        outs = pprint(marked_words(df, [race], ['gender'],['male']))
        mw[race] = outs
    temps = []
    for race in df['gender'].unique():
        temp = pprint(marked_words(df, ['male'], ['gender'],[race]))
        temps.extend(temp)

    seen = Counter(temps).most_common()
    mw['male']=[w for w, c in seen if c == 1]

    all_mw[model]=mw
    for k,v in mw.items():
        print(k,v)
        print()
        if k in df['race'].unique():
            for word in v[:10]:
                count_race_lst[word] = count_race_lst.get(word, 0) + 1
        else:
            print(k)
            for word in v[:10]:
                count_gender_lst[word] = count_gender_lst.get(word, 0) + 1


for model, mw in all_mw.items():
    print(model + ": ")
    for k, v in mw.items():
        print(k + ": ")
        if k in df['race'].unique():
            count_dict = count_race_lst
        else:
            count_dict = count_gender_lst

        for word in v[:10]:
            if count_dict[word] == 6:
                print("\colorbox{myred!100}{" + word + "}", end=", ")
            elif count_dict[word] == 5:
                print("\colorbox{myred!60}{" + word + "}", end=", ")
            elif count_dict[word] == 4:
                print("\colorbox{myred!20}{" + word + "}", end=", ")
            else:
                print(word, end=", ")